In [8]:
import skrf as rf
import numpy as npy
from skrf.constants import *

frequency = rf.Frequency(start=1000, stop=2000, npoints=1, unit='MHz')
ntwkAttn1 = rf.components.Attenuator(5, frequency=frequency)
ntwkAttn2 = rf.components.Attenuator(7, frequency=frequency)
ntwkC = rf.cascade_2port(ntwkAttn1, ntwkAttn2)

sys = rf.MultiNetworkSystem()

# add all the networks to the system
sys.add(ntwkAttn1, 'attn1')
sys.add(ntwkAttn2, 'attn2')

# make all the connections between the elements
sys.connect('attn1', 2, 'attn2', 1)

# set the external port numbers, for this example the reduced network is a two-port network. The following is used to identify port 1 and port 2 of that resulting network.
sys.external_port('attn1', 1, external_port_number=1)
sys.external_port('attn2', 2, external_port_number=2)

# the verify function make sure that all the ports have been asigned and that everything is consistant for the reduction algorithm
(en, e) = sys.verify()
for error in e:
    print(error)
print("Total number of Errors: " + str(en))

# calculate and return the resulting network
ntwk = sys.reduce()
print(ntwk)
print(ntwk.s)
print(ntwk.cs)


Total number of Errors: 0
2-Port Network: '',  1000.0-1000.0 MHz, 1 pts, z0=[50.+0.j 50.+0.j]
[[[0.        +0.j 0.25118864+0.j]
  [0.25118864+0.j 0.        +0.j]]]
[[[1.87562646e-21+0.j 0.00000000e+00+0.j]
  [0.00000000e+00+0.j 1.87562646e-21+0.j]]]
